# Sales Prediction Assessment

© Joshua Olalemi

---
### Project Overview: ML Prediction assessment

Further to the discussion I had with Mr Paul Oluyege of MaonTech, I was asked to build and train a machine-learning prediction system on the sales dataset provided. I am to use the data for the last 3 months' average monthly sales (AMS) number to predict the next month's average monthly sales number.

The further instructions given are:

- Dataset shared must be uploaded to PostgreSQL DB and read from there.
- Use a suitable model, bear in mind the prediction accuracy (i.e Linear regression)
- The output is as follows.
> 1. A job service (to auto-train the model on a scheduled basis  ) or an endpoint (as a manual trigger) to train the prediction model.
> 2. endpoint to get predicted value of sales for next month's future sales.

### Dataset data info

- `region` - the region where the depot is located, i.e SW, NT, SE, LG
- `depot` - the name of the depot i.e ABEOKUTA, also known as `location`
- `item_no` - Product number i.e 10040447, also known as `SKU`.
- `AMS` - Average monthly sales
- `month` - Month number, i.e Jan =1, Feb =2, ..., Dec = 12
- `year` - Operation year. i.e 2022.

### Project Methodology
From the problem statement, I deduced the below frame work for workflow:
#### Explore the Data:
#### Feature Engineering:
#### Modelling:
#### Deployment:

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Data Engineering</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>

<a href=#seven>7. Conclusion and Recommendation</a>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Importing Packages ⚡ |
| :--------------------------- |
| In this section I import, and briefly discuss, the libraries that will be used throughout my analysis and modelling. |

---

In [315]:
#For connecting to postgre
import psycopg2
import sys
sys.path.append('../') #to add the directory containing the config script
import maon_config as config
import pandas as pd
import numpy as np

#For visualization
import plotly.express as pe

#For releasing memory
import gc

#for preprocessing and modelling
from sklearn.model_selection import train_test_split
#for modelling
from sklearn.neighbors._regression import KNeighborsRegressor
from sklearn.ensemble._forest import ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree._classes import ExtraTreeRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import LinearRegression
#for model evaluation
from sklearn.metrics import mean_squared_error, r2_score

#from lazypredict.Supervised import LazyRegressor


<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Loading the data ⚡ |
| :--------------------------- |
| In this section I connected to the postgre server hosting the data and loaded it into a DataFrame. |

---

In [316]:
def connect():
    
    # Set up a connection to the postgres server.
    print("Setting up the postgres server...")
    conn_string = "host="+ config.PGHOST  +" dbname="+ config.PGDATABASE +" user=" + config.PGUSER \
                  +" password="+ config.PGPASSWORD
    #print(conn_string)
    print("\t...completed\nConnecting to the server...")
    conn = psycopg2.connect(conn_string)
    print("\t...Server Connected!")
    print("Importing data...")
    cursor = conn.cursor()
    
    # Create a cursor object
    cursor.execute("SELECT * FROM maontech_dataset")
    data = cursor.fetchall()
    print("\t...data imported!")
    return data

In [317]:
data = connect()
columns = ["region", "depot", "item_no", "tms", "ams", "month", "year"]
df = pd.DataFrame(data, columns=columns)
df.head()
del columns

Setting up the postgres server...
	...completed
Connecting to the server...
	...Server Connected!
Importing data...
	...data imported!


As seen above, our target variable `ams` is not occupying the last column position in the dataset. Let's rearrange the columns to ensure it is the last column there.

In [318]:
cols = [col for col in df.columns if col != 'ams'] + ['ams']

df = df.reindex(columns=cols)
del cols
df.head()

,region,depot,item_no,tms,month,year,ams
0,jags,FUNT,10063228,0.000000,9,2022,0.000000
1,SW,ABEOKUTA,10040447,8.878000,1,2022,4.439000
2,SW,ABEOKUTA,10040447,9.313929,2,2022,4.656964
3,SW,ABEOKUTA,10040447,10.244129,3,2022,5.122065
4,SW,ABEOKUTA,10040447,10.821800,4,2022,5.410900


<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Exploratory data analysis ⚡ |
| :--------------------------- |
| In this section, I performed an in-depth analysis of all the variables in the DataFrame. |

---


In [319]:
#check the dimension of the dataset
df.shape

(4628, 7)

As seen from the above, there are `4,628` entries and `7` features in the given dataset.

In [320]:
#check the information of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4628 entries, 0 to 4627
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   region   4628 non-null   object 
 1   depot    4628 non-null   object 
 2   item_no  4628 non-null   object 
 3   tms      4628 non-null   float64
 4   month    4628 non-null   int64  
 5   year     4628 non-null   int64  
 6   ams      4628 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 253.2+ KB


Surprisingly, there are no null values in the dataset because the `Non-Null Count` is equal to the total entries provided. Furthermore, it is seen that the datatypes of the features are `object`, `float64`, and `int64`. We will need to convert the features with an `object` datatype to numerical, since the ML model can only use numerical datatypes.

In [321]:
#check the number of unique entries in each column
df.nunique()

region        2
depot        21
item_no      23
tms        3115
month        12
year          1
ams        3115
dtype: int64

We can see the count of the unique entries in the features. Let's investigate these unique entries further...

In [322]:
cols = [col for col in df.columns if str(col) not in ["tms", "ams"]]

for col in cols:
    print("----"*20)
    print(f"The counts of the unique entries in the '{col}' column are:")
    print(df[col].value_counts())
    print("...and their total count is: " + str(df[col].nunique()))

--------------------------------------------------------------------------------
The counts of the unique entries in the 'region' column are:
SW      4627
jags       1
Name: region, dtype: int64
...and their total count is: 2
--------------------------------------------------------------------------------
The counts of the unique entries in the 'depot' column are:
ILORIN         252
IBADAN         252
ADO EKITI      252
AKURE          252
BENIN          252
AGBOR          248
WARRI          247
ABEOKUTA       239
SHAGAMU        237
ONDO           234
EKPOMA         228
IKARE AKOKO    228
OSHOGBO        228
IJEBU          227
OKENE          225
ISEYIN         215
OGBOMOSHO      214
LOKOJA         204
SHAKI          203
IWO            190
FUNT             1
Name: depot, dtype: int64
...and their total count is: 21
--------------------------------------------------------------------------------
The counts of the unique entries in the 'item_no' column are:
HIT20NG     240
10056877    240
1

From the above, we can see that the region `jags` has just one (1) entry in the whole dataset. This is quite infinitesimal
and can be deleted to improve the performance of our model.

Also, we can see that the depot in `FUNT` has just one (1) entry. This can also be dropped for reasons stated above.

Finally, the `year` feature has just one (1) unique entry. This definitely won't have any impact on the model. Therefore, we will consider dropping (deleting) the feature.

In [323]:
#check for duplicated entries
df.duplicated().sum()

0

We can see that the dataset has no duplicated entry.

In [324]:
#view 10 random entries
df.sample(10)

,region,depot,item_no,tms,month,year,ams
4550,SW,WARRI,D108881,29.500800,4,2022,14.750400
4626,SW,WARRI,TB00004,0.000000,11,2022,0.000000
2841,SW,LOKOJA,10040447,58.273867,6,2022,29.136933
3230,SW,OGBOMOSHO,RKF20HLK,334.619800,1,2022,167.309900
4208,SW,SHAKI,10063228,0.000000,7,2022,0.000000
1919,SW,IJEBU,PMM20HLK,2.701548,5,2022,1.350774
4563,SW,WARRI,HIT20NG,233.445226,5,2022,116.722613
3420,SW,OKENE,HIT20NG,18.321867,11,2022,9.160933
553,SW,AGBOR,10081184,0.000000,6,2022,0.000000
4118,SW,SHAGAMU,HIT20NG,2.332000,1,2022,1.166000


It appears that the `tms` feature is simply twice (2x) the `ams` feature. This is will give rise to the problem of multicollinearity for our model. To overcome this, we can consider droping the `tms` feature, since `ams` is our target variable.

In [325]:
#check the statistical description of the dataset
df.describe().T

,count,mean,std,min,25%,50%,75%,max
tms,4628.0,105.477907,371.070396,-0.129032,0.0,3.033605,44.059026,5162.244
month,4628.0,6.528522,3.443239,1.000000,4.0,7.000000,10.000000,12.000
year,4628.0,2022.000000,0.000000,2022.000000,2022.0,2022.000000,2022.000000,2022.000
ams,4628.0,52.738954,185.535198,-0.064516,0.0,1.516803,22.029513,2581.122


As seen from the above statistical description of the given dataset, the minimum values of `ams` and `tms` is negative. This is quite wrong because it is practically not obtainable to have a negative sales. We will consider taking the absolute values of these columns.

In [326]:
#Check for seasonality in the target variable
pe.line(df, x='month', y='ams')

There is no obvious seasonality in the given dataset. Therefore, we will only consider a Linear model, not timeseries.

In [327]:
#Release memory
gc.collect()

1200

<a id="four"></a>
## 4. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data engineering ⚡ |
| :--------------------------- |
| In this section I cleaned the dataset, and created new features - as identified in the EDA phase. |

---

As identified during the EDA section, here are some of engineering to be done to the data:

- drop columns: '`year`', '`tms`'
- drop rows: '`jags`' (under the '`region`' column)
- Convert datatypes to numeric: '`region`', '`depot`', and '`item_no`'
- Absolute values: target variable('`ams`')

In [328]:
#drop 'jags' rows
df = df[df['region']=="SW"]
df.head() #view the first 5 entries

,region,depot,item_no,tms,month,year,ams
1,SW,ABEOKUTA,10040447,8.878000,1,2022,4.439000
2,SW,ABEOKUTA,10040447,9.313929,2,2022,4.656964
3,SW,ABEOKUTA,10040447,10.244129,3,2022,5.122065
4,SW,ABEOKUTA,10040447,10.821800,4,2022,5.410900
5,SW,ABEOKUTA,10040447,6.610065,5,2022,3.305032


Since we have subset the dataset to only contain entries where the region is `SW`, it therefore implies that we only have one region left in the dataset. This is of no statistical relevance. We therefore dropped the entire `region` feature.

In [329]:
#drop 'region', 'tms', 'year' features
df.drop(['region', 'tms', 'year'], axis = 1, inplace=True)

df.head() #view the first 5 entries

,depot,item_no,month,ams
1,ABEOKUTA,10040447,1,4.439000
2,ABEOKUTA,10040447,2,4.656964
3,ABEOKUTA,10040447,3,5.122065
4,ABEOKUTA,10040447,4,5.410900
5,ABEOKUTA,10040447,5,3.305032


Since we are more particular about the monthly prediction, it is better we sort the dataset by month.

In [330]:
df = df.sort_values(by='month')

As also identified, we will take the absolute values of the target variable, `ams`;

In [331]:
#get the absolute values of the 'ams' feature
df['ams'] = abs(df['ams'])

#### Dealing with Non-numerical Datatypes

As highlighted during the EDA section, there are some non-numerical columns present in the dataset. We will handle these features using `Pandas`' `get_dummies` function.

In [332]:
#convert data type to numerical
df_dummies = pd.get_dummies(df)

#reindex the columns to make the target variable the last
cols = [col for col in df_dummies.columns if col != 'ams'] + ['ams']
df_dummies = df_dummies.reindex(columns=cols)

#delete col
del col
#view first 5 entries
df_dummies.head()

,month,depot_ABEOKUTA,depot_ADO EKITI,depot_AGBOR,depot_AKURE,depot_BENIN,depot_EKPOMA,depot_IBADAN,depot_IJEBU,depot_IKARE AKOKO,...,item_no_113441,item_no_BHF20HLK,item_no_D108881,item_no_HIT20NG,item_no_PMF20HLK,item_no_PMM20HLK,item_no_RKF20HLK,item_no_RSF20HLK,item_no_TB00004,ams
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.4390
656,1,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,234.9937
3100,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0000
3088,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12.5350
668,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1.0980


<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, I built some regression models that are able to accurately predict the average monthly sales ('ams'). |

---

In [333]:
X_data = df_dummies[[col for col in df_dummies.columns if col != "ams"]]
y_data = df_dummies['ams']

In [334]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                    y_data,
                                                    test_size=0.20,
                                                    random_state=42,
                                                    shuffle=False)

### KNeighborsRegressor Model

In [335]:
#create an object for the model
knr = KNeighborsRegressor()

In [336]:
#train the model
knr.fit(X_train, y_train)

KNeighborsRegressor()

In [337]:
#use the model to predict
knr_y_pred = knr.predict(X_test)

In [338]:
#Evaluate the model
MSE = mean_squared_error(y_test, knr_y_pred)
knr_rmse = np.sqrt(MSE)
knr_rmse

158.36143952682073

In [339]:
#R-squared score
knr_r2 = r2_score(y_test, knr_y_pred)
knr_r2

0.22728617232529957

### ExtraTreesRegressor Model

In [340]:
#create an object for the model
etrs = ExtraTreesRegressor(random_state=42)

In [341]:
#train the model
etrs.fit(X_train, y_train)

ExtraTreesRegressor(random_state=42)

In [342]:
#use the model to predict
etrs_y_pred = etrs.predict(X_test)

In [343]:
#Evaluate the model
MSE = mean_squared_error(y_test, etrs_y_pred)
etrs_rmse = np.sqrt(MSE)
etrs_rmse

60.80745347918957

In [344]:
#R-squared score
etrs_r2 = r2_score(y_test, etrs_y_pred)
etrs_r2

0.8860712231841339

### ExtraTreeRegressor Model

In [345]:
#create an object for the model
etr = ExtraTreesRegressor(random_state=42)

In [346]:
#train the model
etr.fit(X_train, y_train)

ExtraTreesRegressor(random_state=42)

In [347]:
#use the model to predict
etr_y_pred = etr.predict(X_test)

In [348]:
#Evaluate the model
MSE = mean_squared_error(y_test, etr_y_pred)
etr_rmse = np.sqrt(MSE)
etr_rmse

60.80745347918957

In [349]:
#R-squared score
etr_r2 = r2_score(y_test, etr_y_pred)
etr_r2

0.8860712231841339

### RandomForestRegressor Model

In [350]:
#create an object for the model
rfr = RandomForestRegressor(random_state=42)

In [351]:
#train the model
rfr.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [352]:
#use the model to predict
rfr_y_pred = rfr.predict(X_test)

In [353]:
#Evaluate the model
MSE = mean_squared_error(y_test, rfr_y_pred)
rfr_rmse = np.sqrt(MSE)
rfr_rmse

54.196904226222735

In [354]:
#R-squared score
rfr_r2 = r2_score(y_test, rfr_y_pred)
rfr_r2

0.9094957941003892

### XGBRegressor Model

In [355]:
#create an object for the model
xgb = XGBRegressor(random_state=42)

In [356]:
#train the model
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42, ...)

In [357]:
#use the model to predict
xgb_y_pred = xgb.predict(X_test)

In [358]:
#Evaluate the model
MSE = mean_squared_error(y_test, xgb_y_pred)
xgb_rmse = np.sqrt(MSE)
xgb_rmse

59.13151961526546

In [359]:
#R-squared score
xgb_r2 = r2_score(y_test, xgb_y_pred)
xgb_r2

0.8922647354887779

### LinearRegression Model

In [360]:
#create an object for the model
lr = LinearRegression()

In [361]:
#train the model
lr.fit(X_train, y_train)

LinearRegression()

In [362]:
#use the model to predict
lr_y_pred = lr.predict(X_test)

In [363]:
#Evaluate the model
MSE = mean_squared_error(y_test, lr_y_pred)
lr_rmse = np.sqrt(MSE)
lr_rmse

142.89161263507214

In [364]:
#R-squared score
lr_r2 = r2_score(y_test, lr_y_pred)
lr_r2

0.37088031063667937

<a id="six"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model performance ⚡ |
| :--------------------------- |
| In this section, I compared the relative performance of the various trained ML models on a holdout dataset and comment on what model is the best and why. |

---

In [365]:
models = ["KNeighborsRegressor", "ExtraTreesRegressor",
        "ExtraTreeRegressor", "RandomForestRegressor",
        "XGBRegressor", "LinearRegression"]
rmse = [knr_rmse, etrs_rmse, etr_rmse, rfr_rmse, xgb_rmse, lr_rmse]
r2 = [knr_r2, etrs_r2, etr_r2, rfr_r2, xgb_r2, lr_r2]
columns = ["RMSE", "R2_Score"]
model_performance = pd.DataFrame(columns=columns, index=models)
model_performance['RMSE'] = rmse
model_performance["R2_Score"] = r2
model_performance

,RMSE,R2_Score
KNeighborsRegressor,158.361440,0.227286
ExtraTreesRegressor,60.807453,0.886071
ExtraTreeRegressor,60.807453,0.886071
RandomForestRegressor,54.196904,0.909496
XGBRegressor,59.131520,0.892265
LinearRegression,142.891613,0.370880


As seen from the above, the best performing model is `RandomForestRegressor`. This will be implemented in the flask app.

<a id="seven"></a>
## 7. Conclusion and Recommendation
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Conclusion and recommendation ⚡ |
| :--------------------------- |
| In this section, I derived some conclusions on the project and gave some recommendation. |

---

Since the `RandomForestRegressor` appears to be the best performing models among the ones tested, I will proceed to write a python script to run the prediction and deploy it using flask.

Secondly, for the monthly batch prediction, I will explore another endpoint option, which is `streamlit`.

The endpoints will be hosted on the cloud and accessible to anyone, anywhere.

Thank you.